### HT table results for inhibitory types

In [1]:
source("./scale.R")
test <- read.csv("/data/sag_data.csv")
options(warn=-1)

Loading required package: spatstat.data

Loading required package: spatstat.geom

spatstat.geom 3.2-9

Loading required package: spatstat.random

spatstat.random 3.2-3

Loading required package: spatstat.explore

Loading required package: nlme

spatstat.explore 3.2-7

Loading required package: spatstat.model

Loading required package: rpart

spatstat.model 3.2-11

Loading required package: spatstat.linnet

spatstat.linnet 3.1-5


spatstat 3.0-8 
For an introduction to spatstat, type ‘beginner’ 



Attaching package: ‘dplyr’


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
seeds <- sample(1:100,5,replace=FALSE)
seeds

[1] 25 42 69 17 61

In [3]:
library('foreach')
library('doParallel')
cores=detectCores()
cl <- makeCluster(cores[1]-1) #not to overload your computer
registerDoParallel(cl)

Loading required package: iterators

Loading required package: parallel



## HT table calculation

In [4]:
ecdf_fun <- function(x,perc) ecdf(x)(perc)
## Calculate column stats for each type
calc_pp_2d<-function(typ,pp,ps,section){
    nnp <- c()
    nni <- c()
    mnn <- c()
    res<- list()
    for (i in 1:length(pp)){     
        perc<- ecdf_fun(nndist(pp[[i]]), 0.01)
        int<- intensity(pp[[i]])
        mval<- mean(marks(pp[[i]]))
        nnp<-append(nnp,perc)
        nni<-append(nni,int)
        mnn <- append(mnn, mval)
        res[[as.character(section[i])]] = min(ps[,i])    
    }   
    pval <- signif(fisher(unname(unlist(res)))$p,2)
    res[['pval']] = pval
    res[['perc']] = round(mean(nnp)*100,2)
    res[['lam']] = round(mean(nni))
    res[['conf']] = round(mean(mnn),2)
    res[['rm']] = length(ps[,1])
    res[['Cluster']] = typ
    return(res)
}

### Inhibitory clusters and supertypes

In [5]:
df<- read.csv("./temp/filter-inhib.csv")
df1 <- replace(df, is.na(df), 0)
df1[df1<=0.5] <- NA
df1<-df1[rowSums(is.na(df1)) != ncol(df1)-1, ]
row.names(df1) <- NULL
types1 = df1$cluster
types1

[1] "0624 Vip Gaba_1"   "0641 Vip Gaba_5"   "0645 Vip Gaba_6"  
 [4] "0709 Lamp5 Gaba_1" "0740 Pvalb Gaba_3" "0741 Pvalb Gaba_3"
 [7] "0742 Pvalb Gaba_3" "0747 Pvalb Gaba_4" "0754 Pvalb Gaba_8"
[10] "0811 Sst Gaba_13"

In [6]:
sections1<- apply(df1[,-1], 1, function(i) colnames(df1[,-1])[ !is.na(i) ]) ## qualified colns
sections1<- sapply(sections1, function(x){as.numeric(gsub("X.", "", x))})

In [7]:
sups_cluster<-function(types){
    sups<-c()
    for (t in types){
        sup<- unique(test[test$cluster %in% t,]$supertype)
        sups<-append(sups, sup)
    }
    return(sups)
}

In [8]:
df<- read.csv("filter-inhib.csv")
df1 <- replace(df, is.na(df), 0)
df1[df1<=0.5] <- NA
df1<-df1[rowSums(is.na(df1)) != ncol(df1)-1, ]
df1$sups<-sups_cluster(df1$cluster)
df1<-df1[!df1$cluster%in%(c("0741 Pvalb Gaba_3",'0742 Pvalb Gaba_3')), ]
row.names(df1) <- NULL

In [9]:
types2<-df1$sups
sections2<- apply(df1[,-c(1,7)], 1, function(i) colnames(df1[,-c(1,7)])[ !is.na(i) ]) ## qualified colns
sections2<- sapply(sections2, function(x){as.numeric(gsub("X.", "", x))})

In [10]:
types<- c(types1, types2)
sections<-c(sections1, sections2)

In [11]:
df = data.frame(matrix(vector(), 0, 11,
                dimnames=list(c(), c('Cluster','24','23','22','21','20',"pval", "perc", "lam",'conf','rm'))),
                stringsAsFactors=F)
df$Cluster <- as.character(df$Cluster)
df

Cluster,X24,X23,X22,X21,X20,pval,perc,lam,conf,rm
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


In [12]:
set.seed(seeds[1]) 
res1 <- foreach(t = 0:length(types), .combine=function(x,y) bind_rows(as.data.frame(x),as.data.frame(y)),.packages=c('dplyr','spatstat','poolr')) %dopar% {
    if (t == 0){
        q <- df
    }
    else{
        pp<-create_pp(test, NULL,sections[[t]], types[t])
        rm<-min(as.numeric(quantile(do.call(c,lapply(pp, nndist)),0.5)),0.12) 
        ps<-get_pp_2d(pp, rm, FALSE)
        
        ## write into data frame
        q <- calc_pp_2d(types[t],pp,ps,sections[[t]])    
        q
    }
}
res1

Cluster,X24,X23,X22,X21,X20,pval,perc,lam,conf,rm
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0624 Vip Gaba_1,0.010,NA,NA,NA,NA,0.0100,0.00,12,0.86,111
0641 Vip Gaba_5,NA,NA,0.010,NA,NA,0.0100,0.00,17,0.72,111
0645 Vip Gaba_6,0.015,NA,NA,NA,0.080,0.0093,0.00,14,0.82,111
0709 Lamp5 Gaba_1,NA,NA,0.035,NA,NA,0.0350,0.00,40,0.76,72
0740 Pvalb Gaba_3,0.030,0.02,NA,0.835,0.015,0.0027,2.94,84,0.81,50
0741 Pvalb Gaba_3,0.010,NA,NA,NA,NA,0.0100,0.00,66,0.84,59
0742 Pvalb Gaba_3,0.095,0.07,NA,NA,NA,0.0400,0.00,39,0.89,83
0747 Pvalb Gaba_4,NA,NA,NA,NA,0.015,0.0150,0.00,20,0.82,111
0754 Pvalb Gaba_8,NA,NA,NA,0.010,0.040,0.0035,1.67,43,0.86,82


In [13]:
write.csv(res1, "./temp/table-in1.csv",row.names = FALSE)

Iterate the process for 5 different seeds.  

In [1]:
res1<-read.csv("./temp/table-in1.csv")
res2<-read.csv("./temp/table-in2.csv")
res3<-read.csv("./temp/table-in3.csv")
res4<-read.csv("./temp/table-in4.csv")
res5<-read.csv("./temp/table-in5.csv")

In [4]:
res<-Reduce("+", list(res1[,2:6],res2[,2:6],res3[,2:6],res4[,2:6],res5[,2:6])) / 5
res<-data.frame(Cluster=res1$Cluster, res)

pv = c()
for(i in 1:nrow(res)) {
    row <- na.omit(unlist(unname(res[i,2:6])))
    pval <- signif(fisher(row)$p,2)
    pv<-append(pv,pval)
}
res$pval<-pv
res$perc<-res1$perc
res$lam<-res1$lam
res$conf<-res1$conf
res$rm<-res1$rm
res

Cluster,X24,X23,X22,X21,X20,pval,perc,lam,conf,rm
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>
0624 Vip Gaba_1,0.011,NA,NA,NA,NA,0.0110,0.00,12,0.86,111
0641 Vip Gaba_5,NA,NA,0.013,NA,NA,0.0130,0.00,17,0.72,111
0645 Vip Gaba_6,0.025,NA,NA,NA,0.085,0.0150,0.00,14,0.82,111
0709 Lamp5 Gaba_1,NA,NA,0.029,NA,NA,0.0290,0.00,40,0.76,72
0740 Pvalb Gaba_3,0.026,0.039,NA,0.841,0.017,0.0044,2.94,84,0.81,50
0741 Pvalb Gaba_3,0.007,NA,NA,NA,NA,0.0070,0.00,66,0.84,59
0742 Pvalb Gaba_3,0.063,0.040,NA,NA,NA,0.0180,0.00,39,0.89,83
0747 Pvalb Gaba_4,NA,NA,NA,NA,0.017,0.0170,0.00,20,0.82,111
0754 Pvalb Gaba_8,NA,NA,NA,0.008,0.028,0.0021,1.67,43,0.86,82


In [6]:
write.csv(res, "table-in.csv",row.names = FALSE)

In [13]:
p.adjust(res$pval, method="BH")

[1] 0.02314286 0.02314286 0.02314286 0.03480000 0.02314286 0.02314286
 [7] 0.02314286 0.02314286 0.02160000 0.02314286 0.02314286 0.02314286
[13] 0.02314286 0.07650000 0.13000000 0.10588235 0.02160000 0.02314286

**Subclasses**: We do the same tests for subclasses.

In [4]:
types<-c('046 Vip Gaba','049 Lamp5 Gaba','052 Pvalb Gaba','053 Sst Gaba')
sections<- list(c(24,20),c(22),c(24,23,21,20),c(21))
res1 <- foreach(t = 0:length(types), .combine=function(x,y) bind_rows(as.data.frame(x),as.data.frame(y)),.packages=c('dplyr','spatstat','poolr')) %dopar% {
    if (t == 0){
        q <- df
    }
    else{
        pp<-create_pp(test, 'FULL',sections[[t]], types[t])
        rm<-min(as.numeric(quantile(do.call(c,lapply(pp, nndist)),0.5)),0.12) 
        ps<-get_pp_2d(pp, rm, FALSE)

        ## write into data frame
        q <- calc_pp_2d(types[t],pp,ps,sections[[t]])
        q
    }
}